In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from Our_library import *

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/etienne/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/etienne/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/etienne/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/etienne/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

### Caractéristiques de la version 2

* Les solutions sont renormalisées pour globalement rentrer dans l'intervalle [0,1]. Renormalisation linéaire : y = x/(8*moyenne des x) + 3/8.

* Les seconds membres sont rescalés aussi avec StandardScaler()

* Le réseau auquel on aboutit après un peu d'optimisation est constitué de 3 couches : 50 Relu, 1000 Relu, 1 Sigmoïd

* Fonction loss : mean_squared_error

* Nombre d'entraînements : 10


Idées pour améliorer le réseau :

* L'entraîner avec plus de données

* L'entaîner sur les données reduced (gain de temps !)

* Pour l'optimisation sur le nombre de neurones par couche : procéder de façon plus méthodique en testant toutes les combinaisons simultanées (plutôt que faire varier une couche à la fois)

* Essayer de rescaler les solutions autrement (par exemple juste diviser par le max ?)

* Tester d'autres fonctions d'activation ?

# Configuration des données

In [2]:
#importation des données ICI
secondmembres = get_SecondMember("SecondMember_test.pickle")
solutions = get_Solutions("OurSolutions_test.pickle")

FileNotFoundError: [Errno 2] No such file or directory: 'SecondMember_test.pickle'

In [ ]:
#Mise en forme des données
moyen_solutions = np.mean(solutions)
solutions = solutions/(8*moyen_solutions) + 3/8
moyen_solutions = np.mean(solutions)

#écart maximal entre les solutions 
val = (np.max(np.abs(solutions))-np.min(np.abs(solutions)))
print(val)

#Visualisation des solutions
plt.boxplot(solutions, whis = [2.5,97.5])

In [ ]:
#Normalize
scaler = StandardScaler()
secondmembres = scaler.fit_transform(secondmembres)

pb_train, pb_test, sol_train, sol_test = train_test_split(secondmembres, solutions, test_size = 0.2, random_state = 1)

In [ ]:
# pb_train = np.reshape(secondmembres,(832,1,118714))
# pb_test = np.reshape(secondmembres,(1040-832,1,118714))

# Création du modèle

In [ ]:
model = tf.keras.models.Sequential()

########## Add the layers ##########
model.add(tf.keras.layers.Dense(150, activation = "relu"))
model.add(tf.keras.layers.Dense(100, activation = "relu"))
model.add(tf.keras.layers.Dense(1,activation = "sigmoid"))

In [ ]:
########## Define the loss function ##########
model.compile(
        loss = "mean_squared_error",
        optimizer = "sgd",
        metrics = ["mean_squared_error"]
        )

# Entraînement du modèle

In [ ]:
########### Train the model ##########
history = model.fit(pb_train, sol_train, epochs = 10, validation_split = 0.3)

In [ ]:
model.summary()

# Visualisation des performances du modèle

In [ ]:
loss_curve = history.history["loss"]
loss_val_curve = history.history["val_loss"]

plt.plot(loss_curve, label = "Train")
plt.plot(loss_val_curve, label = "Val")
plt.legend(loc = 'upper left')
plt.title("Loss")
plt.show()

### Enorme overfitting !
Idées pour corriger :
* Modifier le nombres de neurones dans chaque couche : fait ci-dessous
* Modifier les fonctions d'activation
* Entraîner avec plus de données

# Évaluation du modèle

In [ ]:
a = model.evaluate(pb_test, sol_test)

# Sauvegarde du modèle

In [ ]:
model.save("test1.h5")

# Recherche du meilleur modèle

In [ ]:
# La 1 ère couche

loss = [];

for i in range(50,1001,50):
    
    model_temporaire = tf.keras.models.Sequential()
    model_temporaire.add(tf.keras.layers.Dense(i, activation = "relu"))
    model_temporaire.add(tf.keras.layers.Dense(100, activation = "relu"))
    model_temporaire.add(tf.keras.layers.Dense(1,activation = "sigmoid"))
    
    model_temporaire.compile(
        loss = "mean_squared_error",
        optimizer = "sgd",
        metrics = ["mean_squared_error"]
        )
    
    history_temporaire = model_temporaire.fit(pb_train, sol_train, epochs = 10, validation_split = 0.3)
    
    loss.append(model_temporaire.evaluate(pb_test, sol_test)[1])

In [ ]:
# L'index de l'érreur la plus petite
print(np.argmin(loss))

#L'érreur la plus petit
print(np.min(loss))

### BILAN : Avec un nombre de neurones choisi un peu au hasard pour la deuxième couche, ce qui marche le mieux c'est 50 neurones pour la première couche

In [ ]:
#Visualisation de résultat
plt.plot(loss)

In [ ]:
# La 2 ème couche

loss = [];

for j in range(50,1001,50):
    
    model_temporaire = tf.keras.models.Sequential()
    model_temporaire.add(tf.keras.layers.Dense(50, activation = "relu"))
    model_temporaire.add(tf.keras.layers.Dense(j, activation = "relu"))
    model_temporaire.add(tf.keras.layers.Dense(1,activation = "sigmoid"))
    
    model_temporaire.compile(
        loss = "mean_squared_error",
        optimizer = "sgd",
        metrics = ["mean_squared_error"]
        )
    
    history_temporaire = model_temporaire.fit(pb_train, sol_train, epochs = 10, validation_split = 0.3)
    
    loss.append(model_temporaire.evaluate(pb_test, sol_test)[1])

### BILAN : Avec 50 neurones pour la première couche, ce qui marche le mieux c'est 1000 neurones pour la deuxième couche

In [ ]:
# L'index de l'érreur la plus petite
print(np.argmin(loss))

#L'érreur la plus petit
print(np.min(loss))

#Visualisation de résultat
plt.plot(loss)

# Création du meilleur modèle

In [ ]:
model_optimise = tf.keras.models.Sequential()

########## Add the layers ##########
model_optimise.add(tf.keras.layers.Dense(50, activation = "relu"))
model_optimise.add(tf.keras.layers.Dense(1000, activation = "relu"))
model_optimise.add(tf.keras.layers.Dense(1,activation = "sigmoid"))

########## Define the loss function ##########
model_optimise.compile(
        loss = "mean_squared_error",
        optimizer = "sgd",
        metrics = ["mean_squared_error"]
        )

########### Train the model ##########
history_optimise = model_optimise.fit(pb_train, sol_train, epochs = 10, validation_split = 0.3)


########### Overfitting ? ###########
loss_curve = history_optimise.history["loss"]
loss_val_curve = history_optimise.history["val_loss"]

plt.plot(loss_curve, label = "Train")
plt.plot(loss_val_curve, label = "Val")
plt.legend(loc = 'upper left')
plt.title("Loss")
plt.show()


a = model_optimise.evaluate(pb_test, sol_test)
model_optimise.save("test_optimise.h5")